## Obtain Data

In this notebook we perform the following steps:
* Establish the first hour of the dataset
* For the first month,
  * Obtain a list of available stations by state
  * Obtain temperature observations from weather stations in the MISO footprint
    * Stations are organized into MISO regions by state boundaries
    * Stations are predominantly clustered in population centers, making many observation redundant
    * There are many lacunae in some stations
  * Obtain the actual hourly MISO Load data and historical Medium-Term Load Forecasts (MTLF)
  * Join Load and MTLF data with weather observations to complete the raw data
  * Persist the data and demonstrate use of dataset wrapper class
* Update the dataset to the current day
* Identify and mitigate lacunae
* Publish the data

### Define Date Ranges

In [2]:
%load_ext autoreload
%autoreload 2

from datetime import datetime, timezone, timedelta
from MISO_data import prevailing_time as est

# beginning of MISO's historical records that include the southern region (zones 8-10)
first_hour = est(2015, 2, 1, 0)
# five hours weather data was lost for 2015-06-17
# first_hour = est(2015, 7, 1, 0)

# latest date with actual load data available is
# l = date.today() - timedelta(days=2)
# last_hour = est(l.year, l.month, l.day, 23)
# instead, fix the date for repeatbility
last_hour = est(2022, 3, 31, 23)

test_split = last_hour - timedelta(days=364, hours=23)
validation_split = test_split - timedelta(days=365)


### Obtain Weather Data for Zones

Source: [Iowa State ASOS Network Downloads](https://mesonet.agron.iastate.edu/request/download.phtml)

`zones` below represents an initial candidate set of weather stations in each
zone. We will examine the data quality from each of the stations in the next
section.

In [19]:
from weather_data import ASOS
from pathlib import Path
from os.path import isfile
import pandas as pd

zones = { 1 : {'MN': ['MSP',  # Minneapolis / St. Paul (STP)
                      'RST',  # Rochester
                      'DYT'], # Duluth
               'ND': ['FAR',  # Fargo
                      'BIS',  # Bismarck
                      'GFK'], # Grand Forks
               'SD': ['ABR'], # Aberdeen
               'WI': ['LSE'], # La Crosse
               'IL': ['SFY']  # Savanna 
              },
          2 : {'WI': ['MSN', 'MKE', 'EAU', 'GRB'],
               'MI': ['ANJ', 'SAW', 'IWD']},
          3 : {'IA': ['DSM', 'CID', 'DVN', 'SUX', 'ALO', 'MCW']}
        }

def download_file_path(zone, state, station):
    zone_data = f"./data/zone_{zone}"
    Path(zone_data).mkdir(exist_ok=True)
    return f'{zone_data}/{state}_{station}.parquet'

asos = ASOS()
def download_station(zone, state, station):
    path = download_file_path(zone, state, station) 
    if isfile(path):
        return pd.read_parquet(path)

    station = asos.get_hourly_observations(station, first_hour, last_hour)
    if station is None:
        print(f'Retrieve {station} failed')
        return None
    station.to_parquet(path)
    return station

from multiprocessing import cpu_count
from joblib import Parallel, delayed
def do_parallel(func, zones):
    parallel = Parallel(n_jobs=cpu_count())
    result = {}
    for zone in zones:
        stations = [(state, station) for state in zones[zone].keys() for station in zones[zone][state]]
        result[zone] = pd.concat(parallel(delayed(func)(zone, state, station) for (state, station) in stations))
    return result

In [20]:
zone_results = do_parallel(download_station, zones)
all_zones = pd.concat(zone_results.values())

### Weather Data Quality

We will look for large lacunae in the original data by examining where a few hours or more were interpolated.

In [21]:
interpolated = all_zones[pd.isna(all_zones['observation_time'])].copy()
f = interpolated.groupby([interpolated.index.date, 'station']).temp.count()
f[f > 4].groupby(level='station').count()

station
ABR     20
ALO     10
ANJ     58
BIS     12
CID     30
DSM      1
DVN     32
DYT     66
EAU     15
FAR     10
GFK     20
GRB     11
IWD    126
LSE     34
MCW     51
MKE      6
MSN     11
MSP      4
RST     12
SAW     11
SFY    123
SUX     50
Name: temp, dtype: int64

In [22]:
f[f > 4].groupby(level='station').max()

station
ABR    18
ALO    23
ANJ    24
BIS    11
CID    23
DSM     6
DVN    24
DYT    24
EAU    24
FAR    16
GFK    11
GRB    11
IWD    24
LSE    24
MCW    24
MKE     9
MSN     8
MSP    11
RST    21
SAW     8
SFY    24
SUX    24
Name: temp, dtype: int64

### Best Station for Each Zone

Based on the above results, we select a single station in each zone with the fewest lacunae in the dataset.

In [24]:
zone1_station = 'MSP'
zone2_station = 'MKE'
zone3_station = 'DSM'

#### Feature Engineering: Temperature Differences

In [25]:
df = asos.get_hourly_observations(zone1_station, first_hour, last_hour)
df1 = df.pivot(columns='station', values='temp')
df2 = df1.shift(periods=1)
df2.iloc[0] = df2.iloc[1] #replace NaN
df2.head()

Fetching https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=tmpf&data=feel&tz=Etc/UTC&format=comma&latlon=yes&year1=2015&month1=1&day1=25&year2=2022&month2=4&day2=9&station=MSP


station,MSP
hour,
2015-02-01 00:00:00-05:00,23.00
2015-02-01 01:00:00-05:00,23.00
2015-02-01 02:00:00-05:00,21.02
2015-02-01 03:00:00-05:00,19.04
2015-02-01 04:00:00-05:00,19.04


In [26]:
zone1 = (df1 - df2)
zone1.mean(axis=1).value_counts()

 0.000000     17223
-0.900000      3470
-2.000000      3387
-1.100000      2848
 2.000000      2737
              ...  
-11.880000        1
-6.840000         1
-15.840000        1
-0.369590         1
 2.022754         1
Length: 549, dtype: int64

### Obtain the Regional MTLF and Actual Load for each Observation Hour

In [ ]:
from rf_al_data import 

Path("./data/mtlf").mkdir(exist_ok=True)
forecast_output_dir = './data/mtlf'
# the actuals aren't available until the next day
actuals = get_daily_rf_al_df(first_hour, last_hour + timedelta(days=2.0), forecast_output_dir)
actuals

### Harmonize Features with Actuals

There are a number of lacunae in the weather observations.

In [ ]:
def mktime_idx(row): 
    return datetime.combine(row['Market Day'].date(), time(row['HourEnding'] - 1), timezone(timedelta(hours = -5)))

actuals['time_idx'] = actuals.apply(mktime_idx, axis = 1)

In [ ]:
actuals.to_parquet('./data/actuals_mtlf.parquet')
actuals

In [ ]:
p = df.pivot(index='valid', columns='station', values='tmpf').dropna()
data = p.join(actuals.set_index('time_idx'), how='inner')

(n, _) = data.shape
(num_weather_observations, _) = df.groupby('valid').count().shape
(num_load_observations, _) = actuals.shape
(num_weather_observations, num_load_observations, len(observation_hours), n)

## Feature Engineering: Business Hours

Can we improve the performance of the model by introducing business hours into the feature set?

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay, BusinessHour

federal_business_days = CustomBusinessDay(calendar=USFederalHolidayCalendar())
bh = BusinessHour()
def is_biz_hour(d):
    return federal_business_days.is_on_offset(d) and bh.is_on_offset(d)
data['IsBusinessHour'] = data.index.to_series().apply(lambda d: 1 if is_biz_hour(d) else 0)